In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} </style>"))

# 케라스 CNN 모델을 이용한 필기체 숫자 인식

## 입력 데이터

In [34]:
import numpy as np
import pandas as pd

In [35]:
import cv2
img = cv2.imread('images/digits.png', cv2.IMREAD_GRAYSCALE)

In [36]:
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
x = np.array(cells)
img_rows, img_cols = 20, 20
X = x.reshape(-1, img_rows, img_cols, 1)
# CNN 이미지처리에서 X는 반드시 4차원이여야 함  
# (갯수, 이미지 2차원, channel)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

In [37]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

### 이미지 파일에 저장하기

파일을 저장하기 위한 폴더를 만듦

In [38]:
import os
path = "./digits"
for i, img_data in enumerate(X): # i는 인덱스
    num_dir = str(np.argmax(y[i]))
    img_name = "%04d.jpg" % i # 정수 네자리.jpg
    img_dir = os.path.join(path, num_dir) # 각 숫자별로 디렉토리 만들고 디렉토리에 각 숫자를 넣을 것
    img_path = os.path.join(path, num_dir, img_name)
    os.makedirs(img_dir, exist_ok=True) # 디렉토리 없으면 만들어라
    cv2.imwrite(img_path, img_data)
    
    # 아래 코드는 진행상황을 보기 위함
    percent = (i/5000)*100 # 숫자 5000개
    progress = int(percent*5/10) * '#'
    print("{:5.1f}%[{:<50s}] {}".format(percent, progress, i), end="\r")

 14.1%[#######                                           ] 703

 20.8%[##########                                        ] 1040

 29.6%[##############                                    ] 1478

 38.9%[###################                               ] 1944

 60.8%[##############################                    ] 3042

 78.2%[#######################################           ] 3911

 92.8%[##############################################    ] 4639

## CNN 레이어 정의

In [39]:
from tensorflow.keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# kernel size는 임의로 변경 가능
# 실제 이미지가 클수록 kernel 사이즈가 크면 더 좋은 결과를 낼 수 있음
# 너무 작은 이미지에 kernel size가 지나치게 크다면 오히려 잘 훈련되지 않음
# 단, kernel size가 클수록 실행 시간은 더 오래 걸릴 것 
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(img_rows, img_cols, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) # 128개에 비해서는 지나치게 많이 하는 편이긴 함..그래도 잘 돌아가긴 돌아간다고
model.add(Dense(10, activation='softmax'))

## 콜백, 훈련 정의, 학습

In [40]:
import tensorflow as tf

In [41]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath="digits-{epoch:03d}-{val_accuracy:.4f}.hdf5",
                             monitor="val_accuracy", save_best_only=True, verbose=1)

### Model Checkpoint Callback
* 학습을 할 때 모델을 체크포인트 할 콜백
* ModelCheckpoint 콜백을 시정하려면 fit() 함수에 validation_split 매개변수를 추가해야 함
    - monitor가 accuracy/loss가 아니라 val_accuracy, val_loss인 경우
    - validation_data=(x_test, y_test) or validation_split=0.2 

* keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    - mode: 'auto'이면 loss인지 accuracy 인지 알아서 설정해줌
    - save_best_only가 False일 경우 모든 경우를 저장하는 것은 부담되는 일. 이 저장하는 주기를 조정해줌

In [42]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_accuracy", patience=5)

### Early Stopping Callback
* 반복문을 돌면서 최고 성능 모델을 찾아낼 때, 초반에 최고 성능의 모델이 찾아져서 그 보다 더 좋은 성능의 모델이 더이상 발견되지 않는 경우 학습을 중단시키는 콜백
* keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
    - min_delta: 이보다 작게 loss가 줄어들거나 accuracy가 증가하면 모델이 개선되었다고 판단하지 않고 Stop
    - patience: 연속에서 patience 이상 모델이 개선되지 않으면 Stop
    - restore_best_weights: save_best_only랑 기능 똑같음..위에서 ModelCheckpoint에서 save_best_only 했다면 굳이 설정하지 않아도 됨

In [43]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])

In [44]:
model.fit(train_X, train_y, validation_data=(test_X, test_y),
          callbacks=[checkpoint, early_stopping],
          batch_size=128, epochs=50, verbose=1)

Train on 3500 samples, validate on 1500 samples
Epoch 1/50
3328/3500 [===========================>..] - ETA: 0s - loss: 12.3559 - accuracy: 0.2404
Epoch 00001: val_accuracy improved from -inf to 0.50867, saving model to digits-001-0.5087.hdf5
3500/3500 [==============================] - 1s 380us/sample - loss: 11.8678 - accuracy: 0.2437 - val_loss: 1.6217 - val_accuracy: 0.5087
Epoch 2/50
3456/3500 [============================>.] - ETA: 0s - loss: 1.8676 - accuracy: 0.3909
Epoch 00002: val_accuracy improved from 0.50867 to 0.70600, saving model to digits-002-0.7060.hdf5
3500/3500 [==============================] - 1s 233us/sample - loss: 1.8581 - accuracy: 0.3937 - val_loss: 1.2169 - val_accuracy: 0.7060
Epoch 3/50
3456/3500 [============================>.] - ETA: 0s - loss: 1.4264 - accuracy: 0.5312
Epoch 00003: val_accuracy improved from 0.70600 to 0.82667, saving model to digits-003-0.8267.hdf5
3500/3500 [==============================] - 1s 227us/sample - loss: 1.4214 - accuracy: 

### 모델 구조 저장하기, 모델 불러오기, 가중치 불러오기

#### 저장

In [13]:
model_json = model.to_json()
with open("digits_model.json", "w") as json_file:
    json_file.write(model_json)

#### 불러오기

In [ ]:
from tensorflow.kras.models import model_from_json
with open("digits_model.json", "r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_jsoin)

#### 가중치 불러오기

In [ ]:
model.load_weights("digits-037-0.9760.hdf5")

## 모델 평가

In [45]:
model.evaluate(test_X, test_y, verbose=0)

[0.14402068857848643, 0.958]

In [46]:
import cv2
img4 = cv2.imread("images/four.png", cv2.IMREAD_GRAYSCALE)
number = img4.reshape(-1, img_rows, img_cols, 1)

In [47]:
number = tf.image.convert_image_dtype(number, tf.float32) # 뭔데 자꾸 오류남..그래서 형변환했음

In [50]:
np.argmax(model.predict(number))

8

## 웹카메라로 숫자 인식하기

In [33]:
cap = cv2.VideoCapture(0)
if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV) # 110보다 작으면 255, 110보다 크면 0 (색반전)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Object의 외곽선 검출
            # hierarchy: object 안에 object가 있는 경우에 대한 정보를 받음
            # 이 코드에서는 바깥에 있는 값만 찾기로..(RETR_EXTERNAL)
            try:
                # 발견한 모든 contour에 대하여
                for i in range(len(contours)):
                    # 숫자를 둘러쌀 박스를 치기 위해
                    contour = contours[i]
                    # 숫자를 둘러싸는 min Circle을 찾음
                    # 중심점 x, y좌표와 반지름을 Return
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if radius > 3:
                        # x_start, x_end / y_start, y_end
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        # 위 좌표들로 직사각형을 그림
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        
                        # 박스 안에 있는 숫자를 인식하기 위해 사각형 영역만 따로 추출
                        roi = bin_img[ys:ye, xs:xe]
                        # 한꺼번에 resize를 하면 정보손실이 크기 때문에 단계별로 resize를 진행함
                        dst = cv2.resize(roi, dsize=(50, 50), interpolation=cv2.INTER_AREA)
                        dst = cv2.resize(dst, dsize=(16, 16), interpolation=cv2.INTER_AREA)
                        # min_pooling : 이미지를 축소할 때, 예를 들어 반으로 감소시키기 위해서 2*2 화소 중 평균값을 하나 추출해서 이미지를 구성함
                        # 왜 16*16? 실제 이미지는 20*20이지만, 어느정도 여백을 주는 것
                        
                        A = np.zeros((20, 20))
                        A[2:-2, 2:-2] = dst[:, :] # 여백 테두리 주고, 가운데에 숫자 이미지 넣음
                        A = A.reshape(-1, 20, 20, 1) # 4차원으로 reshape
                        num = np.argmax(model.predict(A)) 
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
            except Exception as e: pass
            cv2.imshow("Image", bin_img)
            if cv2.waitKey(1) & 0xFF == 27: break # ESC
        else:
            print("No Frame")
            break
else: print("Camera not opened")
    
cap.release()
cv2.destroyAllWindows()

7 <class 'numpy.int64'>
6 <class 'numpy.int64'>
6 <class 'numpy.int64'>
7 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
0 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
4 <class 'numpy.int64'>
4 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
6 <class 'numpy.int64'>
8 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
5 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
0 <class 'numpy.int64'>
8 <class 'numpy.int64'>
3 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
3 <class 'numpy.int64'>
2 <class 'numpy.int64'>
9 <class 'numpy.int64'>
2 <class 'numpy.

5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
1 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
8 <class 'numpy.int64'>
5 <class 'numpy.int64'>
8 <class 'numpy.int64'>
5 <class 'numpy.int64'>
8 <class 'numpy.int64'>
5 <class 'numpy.int64'>
8 <class 'numpy.int64'>
5 <class 'numpy.int64'>
0 <class 'numpy.int64'>
8 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
6 <class 'numpy.int64'>
3 <class 'numpy.int64'>
5 <class 'numpy.int64'>
8 <class 'numpy.

3 <class 'numpy.int64'>
8 <class 'numpy.int64'>
3 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
5 <class 'numpy.int64'>
3 <class 'numpy.int64'>
6 <class 'numpy.int64'>
6 <class 'numpy.int64'>
6 <class 'numpy.int64'>
5 <class 'numpy.int64'>
2 <class 'numpy.int64'>
5 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
5 <class 'numpy.int64'>
5 <class 'numpy.int64'>
2 <class 'numpy.int64'>
8 <class 'numpy.int64'>
6 <class 'numpy.int64'>
3 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
5 <class 'numpy.int64'>
3 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
8 <class 'numpy.

9 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
0 <class 'numpy.int64'>
7 <class 'numpy.int64'>
6 <class 'numpy.int64'>
9 <class 'numpy.int64'>
1 <class 'numpy.int64'>
2 <class 'numpy.int64'>
3 <class 'numpy.int64'>
8 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
7 <class 'numpy.int64'>
6 <class 'numpy.int64'>
9 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
0 <class 'numpy.int64'>
7 <class 'numpy.int64'>
6 <class 'numpy.int64'>
9 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.

3 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
9 <class 'numpy.int64'>
2 <class 'numpy.int64'>
5 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
2 <class 'numpy.int64'>
8 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
6 <class 'numpy.int64'>
9 <class 'numpy.int64'>
3 <class 'numpy.int64'>
9 <class 'numpy.int64'>
2 <class 'numpy.int64'>
1 <class 'numpy.int64'>
5 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.int64'>
2 <class 'numpy.

# CNN 이미지 딥러닝

## 이미지 증강

In [51]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(rotation_range=40, 
                                   width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rotation_range=40,
                                  width_shift_range=0.2, height_shift_range=0.2,
                                  shear_range=0.2, zoom_range=0.2,
                                  horizontal_flip=True, fill_mode='nearest')

In [58]:
import cv2
img0 = cv2.imread('digits/0/0000.jpg', 0) # PIL 이미지

x = img0.reshape(-1, 20, 20, 1)

i = 0
for bath in train_datagen.flow(x, batch_size=1, save_to_dir='digits', save_prefix='zero', save_format='jpeg'):
    i += 1
    if i > 20: break

In [70]:
import cv2
img0 = cv2.imread('digits/4/2080.jpg', 0) # PIL 이미지

x = img0.reshape(-1, 20, 20, 1)

i = 0
for bath in train_datagen.flow(x, batch_size=1, save_to_dir='digits', save_prefix='four', save_format='jpeg'):
    i += 1
    if i > 20: break

In [60]:
from tensorflow.keras.models import model_from_json
with open('digits_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)

In [61]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath="digits-{epoch:03d}-{val_accuracy:.4f}.hdf5",
                             monitor="val_accuracy", save_best_only=True, verbose=1)

In [62]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_accuracy", patience=5)

In [63]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])

In [71]:
model.fit(train_datagen.flow(train_X, train_y),
#           step_per_epoch=len(train_X)/128,
          validation_data=train_datagen.flow(test_X, test_y),
          epochs=500,
          callbacks=[checkpoint, early_stopping],
          verbose=1)

Train for 110 steps, validate for 47 steps
Epoch 1/500
105/110 [===========================>..] - ETA: 0s - loss: 0.9181 - accuracy: 0.7027
Epoch 00001: val_accuracy did not improve from 0.81533
110/110 [==============================] - 2s 17ms/step - loss: 0.9207 - accuracy: 0.7026 - val_loss: 0.6417 - val_accuracy: 0.8107
Epoch 2/500
103/110 [===========================>..] - ETA: 0s - loss: 0.9289 - accuracy: 0.6874
Epoch 00002: val_accuracy did not improve from 0.81533
110/110 [==============================] - 2s 17ms/step - loss: 0.9322 - accuracy: 0.6880 - val_loss: 0.6222 - val_accuracy: 0.8100
Epoch 3/500
105/110 [===========================>..] - ETA: 0s - loss: 0.9511 - accuracy: 0.6916
Epoch 00003: val_accuracy improved from 0.81533 to 0.81733, saving model to digits-003-0.8173.hdf5
110/110 [==============================] - 2s 17ms/step - loss: 0.9507 - accuracy: 0.6903 - val_loss: 0.6117 - val_accuracy: 0.8173
Epoch 4/500
109/110 [============================>.] - ETA: 0

In [72]:
model.evaluate(test_X, test_y, verbose=0)

[0.27880340588092806, 0.91]

In [73]:
import cv2
img4 = cv2.imread("images/four.png", cv2.IMREAD_GRAYSCALE)
number = img4.reshape(-1, img_rows, img_cols, 1)

In [74]:
number = tf.image.convert_image_dtype(number, tf.float32) # 뭔데 자꾸 오류남..그래서 형변환했음

In [75]:
np.argmax(model.predict(number))

1